![header](../figures/DC_SSH_QG_mapping-banner.png)

***
**Authors:**  Datlas & IGE <br>
**Copyright:** 2022 Datlas & IGE <br>
**License:** MIT

<div class="alert alert-block alert-success">
    <h1><center>Understand the diagnostics</center></h1>
    <h5><center>This notebook aims at documenting the available diagnostics & illustrate their uses.</h5> 
</div> 




In [ ]:
import xarray as xr
import hvplot.xarray
import numpy
import sys

sys.path.append('..')

from src.mod_plot import *

<div class="alert alert-info" role="alert">

<h2> Download the data </h2>

</div>

### Load the reference SSH field

In [ ]:
filename_ssh_reference = '../results/dc_ref_2012-10-22-2012-12-02_sample.nc'
dc_ref_sample = xr.open_dataset(filename_ssh_reference)
dc_ref_sample

### Load the reconstructed SSH field

In [ ]:
filename_ssh_reconstruction_regridded = '../results/ssh_reconstruction_regridded_2012-10-22-2012-12-02_jason1.nc'
dc_reconstruction_regridded = xr.open_dataset(filename_ssh_reconstruction_regridded)
dc_reconstruction_regridded

In [ ]:
filename_ssh_reconstruction = '../results/ssh_reconstruction_2012-10-22-2012-12-02_jason1.nc'
dc_reconstruction = xr.open_dataset(filename_ssh_reconstruction)
dc_reconstruction

### Display the reference & reconstructed SSH fields

In [ ]:
ds_concat = xr.concat((dc_ref_sample, dc_reconstruction_regridded), dim='field')
ds_concat['field'] = ["Nature Run", "OI Reconstruction"]
ds_concat

In [ ]:
ds_concat.hvplot.image(x='lon', y='lat', cmap='gist_stern', by='field', width=500, height=400, widget_location='bottom', clabel='SSH [m]', subplots=True, clim=(-0.5, 1))

### Display the difference  SSH_reconstruction minus SSH_reference

In [ ]:
(dc_reconstruction_regridded - dc_ref_sample).hvplot.image(x='lon', y='lat', cmap='coolwarm', width=500, height=400, widget_location='bottom', clabel='dSSH[m]')

### Display spatially averaged daily RMSE score (normalized RMSE)

Shows the temporal stability of the reconstuction. A score of 1 means a perfect reconstruction, a zero score means strong incoherency between the reconstruction & reference fields 

In [ ]:
filename_rmse_t = '../results/rmse_t_ssh_reconstruction_2012-10-22-2012-12-02_jason1.nc'
ds = xr.open_dataset(filename_rmse_t)
ds.rmse_t.hvplot(yaxis='left', title='Spatially averaged daily RMSE', ylim=(0, 1), ylabel='RMSE score')

In [ ]:
dc_reconstruction.nobs.hvplot.area(color='grey', title='# obs in OI')

### Display temporally averaged RMSE

Highlight the regions of large/low mean reconstruction errors

In [ ]:
filename_rmse_xy = '../results/rmse_xy_ssh_reconstruction_2012-10-22-2012-12-02_jason1.nc'
ds = xr.open_dataset(filename_rmse_xy)
ds.rmse_xy.hvplot(x='lon', y='lat', cmap='Reds', width=500, height=400, clabel='RMSE[m]', title='Temporally averaged RMSE')

### Display PSD score

Highlight the average space-time scales that are resolved in the reconstruction. The boundary is delimited by the iso-contour 0.5 (green area in the figure)

In [ ]:
filename_psd = '../results/psd_ssh_reconstruction_2012-10-22-2012-12-02_jason1.nc'
ds = xr.open_dataset(filename_psd)
plot_psd_score(ds)

In [ ]:
ds['freq_time'] = 1./ds['freq_time']
ds['freq_lon'] = 1./ds['freq_lon']
ds.hvplot.quadmesh(x='freq_lon', y='freq_time', z='psd_score', cmap='RdYlGn',
                   clim=(0, 1), logx=False, logy=False, title='PSD-score', width=500, height=400, 
                   clabel='PSD score', xlabel='wavelenght (degree)', ylabel='wavelenght (days)')